In [40]:
## CHECKPOINT #2

# There are three objectives for Checkpoint #2: 
#   - identify relevant metrics (2+) for assessing classifier accuracy
#   - build a custom Neural Network:
#       * with at least one input layer, one hidden layer, one output layer
#   - initial analysis of this custom model
#       * which would require a complete run through of trainning, validating 
#         testing

In [41]:
# FIRST OBJECTIVE

# Choosing the relevant metrics is up to you. There are few things for things
# for you to consider.

# In choosing relevant metrics, I recommend considering at least one ACCURACY
# METRIC AND at leat one LOSS FUNCTION! 
# 1) measure of accuracy: 
#   sci-kit learn's page on this could be useful for you:
#   https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics
#   there are many possible metrics to choose from: accuracy score, F1 score, 
#   precision-recall curve, etc
# 2) loss functions:
#   In class, we've seen Negative Log Loss, which works well for a binary
#   or a multi-class classification. There is also cross-entropy, which is
#   widely used for binary classifcation. There are other loss functions, too!
#   This blog post is a good starting point: https://neptune.ai/blog/pytorch-loss-functions

# CONSIDERATION: though this objective is listed first, I recommend coming to this
# after you built your model. As you build the train/val/test process, assessing
# accuracy and loss will be done pretty simply, often with a call of a single
# function/method for each metric. Start with the metrics you've seen in class
# and then consider branching out! 

In [42]:
# SECOND OBJECTIVE

# Building a custom neural network! The exciting part <3

# FYI: HW4 will be a great introduction to using PyTorch to build a simple
# neural network.

In [43]:
!pip install torchvision 

In [44]:
from torchvision.io import read_image
from silent_p.process.data_module import CustomImageDataset, transforms
import torchvision.transforms as T

ModuleNotFoundError: No module named 'silent_p'

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import our libraries
import torch 
import torch.nn as nn # basic building block for neural neteorks
import torch.nn.functional as F # import convolution functions like Relu
import torch.optim as optim # optimzer

In [ ]:
# 1: make your model

In [126]:
# MY MODEL EXPECTS AN 512 x 512 IMAGE with dtype = float32

class CustomNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        
        # inspire by Turing award winning LeCun, Bengio and Hinton's paper from 1998
        # https://ieeexplore.ieee.org/document/726791 (cited more than 25,000 times!!!!!!!!!)
        # code from https://blog.paperspace.com/writing-lenet5-from-scratch-in-python/ 
        self.LeNet = nn.Sequential(     
            # convolutional layers            
            nn.Sequential(                                            # FIRST LAYER: (INPUT LAYER)
              nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),    # CONVOLUTION 
              nn.BatchNorm2d(6),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size = 2, stride = 2)),           # POOLING
            nn.Sequential(                                            # SECOND LAYER: HIDDEN LAYER 1
              nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),   # CONVOLUTION 
              nn.BatchNorm2d(16),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size = 2, stride = 2)),             # POOLING
            #fully connected layers
            nn.Flatten(),
            nn.Linear(1250448, 120),                                   # THIRD LAYER: LINEAR YEAR, HIDDEN LAYER 2 changed from 250000
            nn.ReLU(),                                                # HIDDEN LAYER's ACTIVATION FUNCION
            nn.Linear(120, 84),                                       # FOURTH LAYER: LINEAR YEAR, HIDDEN LAYER 3
            nn.ReLU(),                                                # HIDDEN LAYER's ACTIVATION FUNCION
            # output layer
            nn.Linear(84, 2)                                          # OUTPUT LAYER
        )

    def forward(self, x):
        out = self.LeNet(x)
        return out


In [47]:
# RESOURCE FOR CHOOSING ACTIVATION FUNCTIONS
# https://towardsdatascience.com/how-to-choose-the-right-activation-function-for-neural-networks-3941ff0e6f9c

# RESOURCES ON CONVOLUTION: 
# NOTE: not necessary to implemenent a convolutional layer. this is helpful
# IF you want to implement your own costum convolutional layer!
# http://www.songho.ca/dsp/convolution/convolution.html#convolution_2d
# http://www.songho.ca/dsp/convolution/convolution2d_example.html

In [127]:
model = CustomNeuralNetwork()

In [124]:
# 2: get your dataloaders from the first checkpoint!
# train_loader = CustomImageDataset("data/output/train.csv", "data/train/", transforms)
# val_loader = CustomImageDataset("data/output/val.csv", "data/val/", transforms)
test_loader = CustomImageDataset("data/output/test.csv", "data/test/", transforms)

torch.Size([1, 240])

In [133]:
inputs, labels = test_loader[0]
inputs.size()
test = int('NORMAL')
torch.tensor(0, dtype=torch.float32) 

# Image features
# Before forward: 1277760 (968 * 1320; height x width) 
# First Convultion layer: adds 625176 (1902936)
# Second layer: pools new features together: removes 652488 features (1250448)
# Final Total before linear tranformation: 1250448 



/Users/jackgibson/opt/anaconda3/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


ValueError: invalid literal for int() with base 10: 'NORMAL'

In [128]:
# 3: Define a Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [148]:
# 4: Train and validate the network
EPOCHS = 1

train_losses = []
train_accuracies = []
val_losses = []
val_accuarcies = []

for _ in range(EPOCHS):  # loop over the dataset multiple times

    # TRAIN
    # Make sure gradient tracking is on, and do a pass over the data
    model.train()
    running_loss = 0.0
    for i in range(len(test_loader)):
      # get the inputs; data is a list of [inputs, labels]
      inputs, labels = test_loader[i]          # NOTE: depending on how you implemented your dataset class's __getitem__ it could be labels, inputs

      if labels == 'NORMAL':
        test_label = torch.tensor([[0, 1]], dtype=torch.float64)
      else:
        test_label = torch.tensor([[1, 0]], dtype=torch.float64) 

    
      inputs = inputs.type(torch.float32)
      inputs.unsqueeze_(0) 

 
      # zero the parameter gradients
      optimizer.zero_grad()

      # forward + backward + optimize
      outputs = model(inputs)
      print('output', outputs)
      print(test_label.size())
      loss = criterion(outputs, test_label)
      loss.backward()
      optimizer.step()

      # keep track of the loss
      running_loss += loss.item()

      # ALSO CALCULATE YOUR ACCURACY METRIC
      
    avg_train_loss = running_loss / (i + 1)     # i + 1 gives us the total number of batches in train dataloader
    # CALCULATE AVERAGE ACCURACY METRIC
    avg_train_loss = None
    train_losses.append(avg_train_loss)
    # train_accuracies.append(avg_train_acc)

    #VALIDATE
    # in the validation part, we don't want to keep track of the gradients 
    model.eval()            
    
    # implement a similar loop!
    # but you can leave out loss.backward()

output tensor([[ 40.3451, -60.1660]], grad_fn=<AddmmBackward0>)
torch.Size([1, 2])
output tensor([[-54.1842,  49.7730]], grad_fn=<AddmmBackward0>)
torch.Size([1, 2])
output tensor([[-67.5330,  64.0472]], grad_fn=<AddmmBackward0>)
torch.Size([1, 2])
output tensor([[-141.3533,  133.5037]], grad_fn=<AddmmBackward0>)
torch.Size([1, 2])
output tensor([[-133.8779,  126.6736]], grad_fn=<AddmmBackward0>)
torch.Size([1, 2])
output tensor([[-150.0403,  142.0678]], grad_fn=<AddmmBackward0>)
torch.Size([1, 2])
output tensor([[-177.9013,  169.1563]], grad_fn=<AddmmBackward0>)
torch.Size([1, 2])
output tensor([[-215.1180,  205.5584]], grad_fn=<AddmmBackward0>)
torch.Size([1, 2])
output tensor([[-219.4533,  210.0621]], grad_fn=<AddmmBackward0>)
torch.Size([1, 2])
output tensor([[-329.3304,  315.4195]], grad_fn=<AddmmBackward0>)
torch.Size([1, 2])
output tensor([[-378.4672,  362.6959]], grad_fn=<AddmmBackward0>)
torch.Size([1, 2])
output tensor([[-410.0830,  393.5332]], grad_fn=<AddmmBackward0>)
torch

In [149]:
running_loss

1317.087062097516

In [ ]:
# 5: Test!

# FOR TESTING YOU DON'T HAVE TO ITERATE OVER MULTIPLE EPOCHS
# JUST ONE PASS OVER THE TEST DATALOADER!


In [ ]:
# 6: ANAYLZE (i.e. 3RD OBJECTIVE)

# YOU CAN MAKE GRAPHS of TRAIN AND VAL LOSSES OVER EPOCHS, etc!
# YOU CAN ALSO DO MULTIPLE TRAININGS, CHOOSING A DIFFERENT LOSS FUNCTION
# FOR EACH TRAINING RUN, AND THEN YOU COULD COMPARE HOW WHICH LOSS FUNCTION
# LEADS TO THE BEST LOSSES OR BEST ACCURACIES

# ALSO YOU COULD TRAIN USING DIFFERENT OPTIMIZERS!

# SO MUCH YOU COULD DO!